# Descargar datos de la versión 3 de WaPOR.

[![](https://raw.githubusercontent.com//wateraccounting/WaPORMOOC/main/images/colab-badge.png)](https://colab.research.google.com/github/wateraccounting/WaPORMOOC/blob/main/1_WaPOR_download_colab/Download_WaPORv3_Data.ipynb?target="_blank")

Accede directamente al Notebook [aquí](https://colab.research.google.com/github/wateraccounting/WaPORMOOC/blob/main/1_WaPOR_download_colab/Download_WaPORv3_Data.ipynb?target="_blank")

En este cuaderno utilizaremos el [paquete wapordl](https://bitbucket.org/cioapps/wapordl/src/main/) puesto a disposición por la FAO para descargar datos de WaPOR v3 y descargar archivos a su computadora local. Este breve [video](https://www.youtube.com/watch?v=ymBM_EVPEMQ) explica cómo utilizar este script. Para obtener más información sobre cómo utilizar los datos descargados de WaPOR v3, consulte el [Curso en línea masivo y abierto sobre introducción a WaPORv3](https://ocw.un-ihe.org/course/view.php?id=263).

   
Los pasos son los siguientes:

0. Instalar y cargar los paquetes necesarios
1. Cargar un archivo geojson para definir el límite del área de interés
2. Utilizar el paquete wapordl para descargar archivos ráster para cada variable
3. Comprimir los archivos ráster y descargarlos a una carpeta local

Este script se puede utilizar sin [montar](https://colab.research.google.com/notebooks/snippets/drive.ipynb) Google Colab con su Google Drive. Si desea utilizar archivos almacenados en su Google Drive o guardar los archivos allí, primero debe montar el script.

**NOTA**: Este script permite descargar datos WaPOR en archivos TIFF separados utilizando la convención de nombres WaPOR, eliminando el factor de escala y creando datos en las unidades según la escala temporal de los datos (por ejemplo, los datos decadales se convierten de mm/día a mm/dekade). Para obtener más información sobre los formatos de datos WaPOR, consulte este [video](https://www.youtube.com/watch?v=nm5IAy4MHg4). La eliminación del factor de escala aumenta el tamaño del archivo; para usuarios más avanzados, consulte el [paquete wapordl](https://bitbucket.org/cioapps/wapordl/src/main/) creado por la FAO para obtener más formas de descargar datos WaPORv3 (incluida una opción para descargar 1 archivo con valores escalados en múltiples bandas para ahorrar espacio; tenga en cuenta que debe asegurarse de que se aplique el factor de escala al utilizar dichos datos).

 © 2024 IHE Delft Licenced under CC BY SA Creative Commons



---



## **Paso 0: instala y carga los paquetes necesarios**
A modo de comprobación, también te informamos la versión de wapordl que estás usando (para informar un problema)

In [1]:
%%capture
!pip install --upgrade wapordl --quiet


In [ ]:
import wapordl
print(wapordl.__version__)

#### Importar y cargar los paquetes necesarios

In [3]:
# from osgeo import gdal
import glob
import os
from wapordl import wapor_map
from wapordl import wapor_ts



---



## **Paso 1: cargue un archivo geojson para definir el límite del área de interés**
El área de interés se puede definir proporcionando un archivo geojson o mediante un cuadro delimitador (coordenadas de la extensión del área de interés). En este paso, puede cargar su archivo geojson. Si decide utilizar el enfoque del cuadro delimitador, puede omitir este paso.

*NOTA*: wapordl solo admite el formato de archivo geojson en este momento. Si tiene un archivo en otro formato, puede utilizar QGIS para exportarlo como geojson.



In [ ]:
#To upload file.
from google.colab import files
uploaded = files.upload()

Si el script no se ejecuta, verifique si el geojson está en formato 2D. (Shapely actualmente no admite geometrías 3D, por lo que la solución alternativa es [convertir sus geometrías 3D a 2D](https://gis.stackexchange.com/questions/255540/convert-3d-shapefile-to-2d-in-qgis)). Para usuarios más avanzados, también puede vincular el script a un archivo almacenado en su Google Drive, pero luego debe montar su script primero.



---



## **Paso 2: use el paquete wapordl para descargar archivos ráster para cada variable**

Para usar el paquete wapordl para descargar datos de WaPOR, debe 1) definir el área de interés (ya sea en un archivo geojson o en un cuadro delimitador definido como [xmin, ymin, xmax, ymax]), 2) las variables a descargar y el período de tiempo, y 3) la carpeta donde guardar los archivos descargados.

Las variables que se pueden descargar incluyen AETI (evapotranspiración e intercepción reales), T (transpiración), I (intercepción), E (evaporación), NPP (producto primario neto), PCP (precipitación), RET (evapotranspiración de referencia) y RSM (humedad relativa del suelo). Hay 3 niveles de datos de WaPOR (L1, L2 y L3) relacionados con la resolución de los datos y en términos de resolución temporal, diario (E), decadal (D), mensual (M) y anual (A). Consulte en el [portal WaPOR](https://data.apps.fao.org/wapor/?lang=es) en qué resolución temporal están disponibles los datos que desea descargar.

Consulte la **[descripción general de WaPOR](https://www.fao.org/in-action/remote-sensing-for-water-productivity/wapor-data/es)** para obtener más información sobre los datos de WaPOR, su cobertura espacial y temporal y el contenido de la base de datos. Para conocer la convención de nomenclatura de las variables de WaPOR, consulte la imagen siguiente.

![](https://raw.githubusercontent.com/wateraccounting/WaPORMOOC/main/images/WaPOR%20naming%20conventions_update.jpg)

In [ ]:
# 1) Definir región (eliminar el comentario (#) de la opción que se desea utilizar)
# Definir región mediante un archivo geojson (si se utiliza un archivo almacenado en Google Drive, 
# agregar la ruta relativa aquí)
region = r"/content/Bekaa.geojson"

# definir región en el cuadro delimitador
# region = [39.17, 8.32, 39.37, 8.50] # [xmin, ymin, xmax, ymax]

# 2) definir las variables y el periodo de tiempo para descargar
variables = ["L2-AETI-A"] #, "L3-T-D","L3-NPP-D"]
period = ["2018-01-01", "2023-12-31"]
overview = "NONE"

for var in variables:

  # 3) Define una carpeta para guardar los rásteres descargados (esto creará una carpeta temporal 
  #llamada salida, si quieres que los datos se almacenen en otro lugar cambia la ruta, 
  # esto requiere montar el script colab)
  folder = r"/content/output"+f"/{var}"
  if not os.path.exists(folder):
          os.makedirs(folder)

  if('-E' in var):
    unit = "day"
  elif('-D' in var):
    unit = "dekad"
  elif('-M' in var):
    unit = "month"
  elif ('-A' in var):
    unit = "year"
  else:
    unit = "none"

  fps = wapor_map(region, var, period, folder, separate_unscale = True, unit_conversion = unit)


Los archivos raster se encuentran en la carpeta de salida y se pueden descargar manualmente. También puede ir al paso 3 para utilizar un script para la descarga.
---



## **Paso 3: comprima los archivos raster y descárguelos en una carpeta local**

Este script comprimirá el contenido de la carpeta de salida y descargará el archivo comprimido en nuestra computadora local ejecutando la siguiente celda.

*NOTA*: actualice las extensiones si ha cambiado la ubicación del archivo de salida.

In [ ]:
!zip -r /content/data.zip /content/output
from google.colab import files
files.download(r'/content/data.zip')


In [ ]:
# # if you want to delete a folder use the code below.
# !rm -rf /content/output
# !rm -rf /content/tifs
# !rm -rf /content/data.zip